In [1]:
import gc
from all_stand_var import conv_dict, used_cols
from all_own_funct import memory_downscale,memory_upscale
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from seaborn import heatmap
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import kurtosis
from matplotlib.backends.backend_pdf import PdfPages
import keras.backend as K
from RNN_LTSM_CHD import return_loaded_model
import pickle
import locale
import tables

from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix

locale.setlocale(locale.LC_ALL, 'fr_FR')
# Create ouput folder
output_folder = os.path.join(os.getcwd(), 'Results_CHD_v5','final')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# REad in data chunkwise
used_cols = ['pat_hosp_id', 'pat_bd', 'pat_datetime', 'OK_datum', 'mon_rr', 'mon_hr', 'mon_sat',
              'mon_etco2', 'vent_m_fio2', 'vent_m_ppeak','vent_m_peep',
             'mon_ibp_mean','pat_weight_act','vent_m_rr', 'vent_m_tv_exp']
dtype_dict={'vent_cat': 'category','vent_machine':'category','vent_mode':'category'}
df_raw=pd.read_csv(r'data\CHD_between.csv',delimiter=';',decimal=',',converters=conv_dict,usecols=used_cols,dtype=dtype_dict,dayfirst=True,parse_dates=['pat_bd','pat_datetime','OK_datum'],na_values=['NULL','null', 'Null','nUll','nuLl','nulL'],chunksize=15000000)


import LR_build_CHD as pp
df_list=[]

for chunk in df_raw:
    df = pp.data_pp_function(chunk)
    df_list.append(df)
    del df

df=pd.concat(df_list,sort=False,ignore_index=False)
df['Adnum'] = df.groupby(['pat_hosp_id', 'OK_datum'], sort=False, as_index=False).ngroup()

df.sort_values(['pat_datetime'], inplace=True)
df=memory_downscale(df)

df.to_hdf(os.path.join(output_folder, 'processed_df.h5'),key='df',mode='w',format='table', complib='zlib', complevel=9)

C:\Users\berend\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)
c:\Users\berend\Documents\Python_Scripts\Patient_Selection\LR_build_CHD.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pat_datetime_temp'] = pd.to_datetime(df['pat_datetime']).dt.date
c:\Users\berend\Documents\Python_Scripts\Patient_Se